## Project – Phase 2 Group 07 Hugo Rocha

In this part it was transformed some data before being loaded by adding a known vocabularies FOAF. And it was enriched the triplestore with external data from dbpedia.

Finaly it was defined the RDFS and RDFS+ rules

For this part of the project it was decided that the examples were reduced to 50 per csv file for the turtle file to occupy less memory


## Imports

In [1]:
import csv
from csv import DictReader
from rdflib import Dataset, URIRef, Literal, Namespace, RDF, RDFS, OWL, XSD, Graph
from rdflib import Graph, URIRef, RDF, Literal
from rdflib.namespace import FOAF, XSD, RDFS
import pandas as pd
from iribaker import to_iri

## Players

In [2]:
data = pd.read_csv('players.csv')
data["player_id"]  = data['PLAYER_ID'].apply(lambda x: str(x))
data["season"]  = data['SEASON'].apply(lambda x: str(x))
data["roster_id"] = data["player_id"]  + "_" + data["season"]
del data["player_id"]
del data["season"]

#NEW only get 50 players (just for the example of the project)
data = data.head(50)
data.to_csv('players.csv')

pls = Namespace("http://www.di.fc.ul.pt/~nba/player#")
tms = Namespace("http://www.di.fc.ul.pt/~nba/team#")
gms = Namespace("http://www.di.fc.ul.pt/~nba/games#")
foaf = Namespace("http://xmlns.com/foaf/0.1/")
dbp = Namespace("http://dbpedia.org/resource/")
owl = Namespace("http://www.w3.org/2002/07/owl#")
schema = Namespace("http://schema.org/")

graph=Graph()
graph.bind("tms", tms)
graph.bind("pls", pls)
graph.bind("foaf", FOAF)
graph.bind("dbp", dbp)
graph.bind("owl", OWL)
graph.bind("gms", gms)
graph.bind("schema", schema)


team_file = "players.csv"
with open(team_file,'r') as csvfile:
    players_contents = [{k: v for k, v in row.items()}
        for row in csv.DictReader(csvfile, skipinitialspace=True, quotechar='"', delimiter=',')]

for p in players_contents:
    rid = URIRef(pls+p["roster_id"])
    pid = URIRef(pls+p["PLAYER_ID"])
    tid  = URIRef(tms+p["TEAM_ID"])
    Season=Literal(p["SEASON"],datatype=XSD.int)
     
    #NEW 
    #adding known vocabularies FOAF    
    graph.add((rid, RDF.type, FOAF.Person))
    player_name =  Literal(p["PLAYER_NAME"], lang='en') 
    graph.add((rid, FOAF.name, player_name))
    
    #Enrich triplestore with external data from dbpedia
    dbp_resourse = URIRef(to_iri(dbp+p["PLAYER_NAME"])) 
    graph.add((rid, RDF.type, dbp_resourse))
    
    graph.add((rid, RDF.type, pls.Player))
    graph.add((rid, pls.player_id, pid))
    graph.add((rid, tms.playsIn, tid))
    graph.add((rid, pls.season, Season))
    graph.add((rid, pls.Name, dbp_resourse))
    


## TEAMS

In [3]:
team_file = "teams.csv"
with open(team_file,'r') as csvfile:
    team_contents = [{k: v for k, v in row.items()}
        for row in csv.DictReader(csvfile, skipinitialspace=True, quotechar='"', delimiter=',')]

tms = Namespace("http://www.di.fc.ul.pt/~nba/team#")
graph.bind("tms", tms)

for s in team_contents:
    tid= URIRef(tms+s["TEAM_ID"])
    lid= URIRef(tms+s["LEAGUE_ID"])
    Min_year=Literal(s["MIN_YEAR"],datatype=XSD.int)
    Max_year=Literal(s["MAX_YEAR"],datatype=XSD.int)
    abre = Literal(s['ABBREVIATION'], datatype=XSD['string'])
    nick = URIRef(to_iri(tms+s["NICKNAME"]))
    nick_name =  Literal(s["NICKNAME"], lang='en')
    year_f=Literal(s["YEARFOUNDED"],datatype=XSD.int)
    arenacapacity=Literal(s["ARENACAPACITY"],datatype=XSD.int)
    
    #NEW
    #adding known vocabularies FOAF
    graph.add((tid, FOAF.name, Literal(s["CITY"] + '_' +s["NICKNAME"], lang="en")))
    
    #Enrich triplestore with external data from dbpedia
    dbp_resourse = URIRef(to_iri(dbp+s["CITY"] + '_' +s["NICKNAME"])) 
    graph.add((tid, RDF.type, dbp_resourse))
    
    Arena_dbp = URIRef(to_iri(dbp+s["ARENA"])) 
    graph.add((tid, tms.arena, Arena_dbp))
    
    City_dbp = URIRef(to_iri(dbp+s["CITY"])) 
    graph.add((tid, tms.city, City_dbp))
    
    Nick_dbp = URIRef(to_iri(dbp+s["NICKNAME"])) 
    graph.add((tid, tms.nickname, Nick_dbp))
    
    

    
    graph.add((tid, RDF.type, tms.Team))
    graph.add((tid, tms.team_id, tid))
    graph.add((tid, tms.league, lid))
    graph.add((tid, tms.min_year, Min_year))
    graph.add((tid, tms.max_year, Max_year))
    graph.add((tid, tms.abbreviation, abre))
    graph.add((tid, tms.year_found, year_f))
    graph.add((tid, tms.arena_capacity, arenacapacity))
    

## Ranking

In [4]:
data = pd.read_csv('ranking.csv')
data["team_idd"]  = data['TEAM_ID'].apply(lambda x: str(x))
data["date"]  = data['STANDINGSDATE'].apply(lambda x: str(x))
data["ranking_id"] = data["team_idd"]  + "_"+ data["date"]
del data["team_idd"]
del data["date"]
data = data.head(50)
data.to_csv('ranking.csv')


rks = Namespace("http://www.di.fc.ul.pt/~nba/ranking#")
graph.bind("rks", rks)

rank_file = "ranking.csv"
with open(rank_file,'r') as csvfile:
    ranking_contents = [{k: v for k, v in row.items()}
        for row in csv.DictReader(csvfile, skipinitialspace=True, quotechar='"', delimiter=',')]


for r in ranking_contents:
       
    rank_id = URIRef(rks+r["ranking_id"])
    tid= URIRef(tms+r["TEAM_ID"]) 
    Season=Literal(r["SEASON_ID"],datatype=XSD.int)
    Date=Literal(r["STANDINGSDATE"],datatype=XSD.date)
    Games=Literal(r["G"],datatype=XSD.int)
    Wins=Literal(r["W"],datatype=XSD.int)
    Loses=Literal(r["L"],datatype=XSD.int)
    WL=Literal(r["W_PCT"],datatype=XSD.decimal)
    
    #NEW
    conference_dbp = URIRef(to_iri(dbp+r["CONFERENCE"]))
    graph.add((rank_id, rks.Conference, conference_dbp))
    team_dbp = URIRef(to_iri(dbp+r["TEAM"])) 
    graph.add((rank_id, rks.Team, team_dbp))
    
    
    graph.add((rank_id, RDF.type, rks.Ranking))
    graph.add((rank_id, tms.team_ranking, tid))
    graph.add((rank_id, rks.season, Season))
    graph.add((rank_id, rks.date, Date))
    graph.add((rank_id, rks.games, Games))
    graph.add((rank_id, rks.wins, Wins))
    graph.add((rank_id, rks.loses, Loses))
    graph.add((rank_id, rks.W_PCT, WL))

## Game Details

In [5]:
gdt = Namespace("http://www.di.fc.ul.pt/~nba/games_details#")
graph.bind("gdt", gdt)

data = pd.read_csv('games_details.csv')

data["game_id"]  = data['GAME_ID'].apply(lambda x: str(x))
data["player_id"]  = data['PLAYER_ID'].apply(lambda x: str(x))
data["player_game"] = data["game_id"]  + "_" + data["player_id"]
del data["game_id"]
del data["player_id"]
data = data.head(50)
data.to_csv('games_details.csv')

rank_file = "games_details.csv"
with open(rank_file,'r') as csvfile:
    game_details_contents = [{k: v for k, v in row.items()}
        for row in csv.DictReader(csvfile, skipinitialspace=True, quotechar='"', delimiter=',')]


for g in game_details_contents:
       
    player_game_id = URIRef(gdt+g["player_game"])
    gid= URIRef(gms+g["GAME_ID"]) 
    tid= URIRef(tms+g["TEAM_ID"]) 
    abre = Literal(g['TEAM_ABBREVIATION'], datatype=XSD['string'])
    pid= URIRef(pls+g["PLAYER_ID"])  
       
    #NEW 
    player_name =  Literal(g["PLAYER_NAME"], lang='en') 
    graph.add((player_game_id, FOAF.name, player_name))
    dbp_resourse = URIRef(to_iri(dbp+g["PLAYER_NAME"])) 
    graph.add((player_game_id, pls.Name, dbp_resourse))
    City_dbp = URIRef(to_iri(dbp+g["TEAM_CITY"]))
    graph.add((player_game_id, tms.city, City_dbp))
    
    graph.add((player_game_id, RDF.type, gdt.Game_Details))
    graph.add((player_game_id, gms.game, gid))
    graph.add((player_game_id, tms.team, tid))
    graph.add((player_game_id, tms.abbreviation, abre))
    graph.add((player_game_id, pls.player_id, pid))
    

## GAMES 
nothing was altered

In [6]:
data = pd.read_csv('games.csv')
data = data.head(50)
data.to_csv('games.csv')

gms = Namespace("http://www.di.fc.ul.pt/~nba/games#")
graph.bind("gms", gms)

rank_file = "games.csv"
with open(rank_file,'r') as csvfile:
    game_contents = [{k: v for k, v in row.items()}
        for row in csv.DictReader(csvfile, skipinitialspace=True, quotechar='"', delimiter=',')]

     
for g in game_contents:
       
    game_id = URIRef(gms+g["GAME_ID"])  
    Game_Date=Literal(g["GAME_DATE_EST"],datatype=XSD.date)
    Season=Literal(g["SEASON"],datatype=XSD.int)
    Home= URIRef(tms+g["HOME_TEAM_ID"]) 
    Visitor= URIRef(tms+g["VISITOR_TEAM_ID"])
    points_home=Literal(g["PTS_home"],datatype=XSD.int)
    points_away=Literal(g["PTS_away"],datatype=XSD.int)
    
    graph.add((game_id, RDF.type, gms.Game))
    graph.add((game_id, gms.game_id, game_id))
    graph.add((game_id, gms.date, Game_Date))
    graph.add((game_id, gms.season, Season))
    graph.add((game_id, tms.home_team, Home))
    graph.add((game_id, tms.visitor_team, Visitor))
    graph.add((game_id, gms.Points_home, points_home))
    graph.add((game_id, gms.Points_away, points_away))
    

## RDFS and RDFS+ rules
Some rules that were possible to define

In [7]:
#PLAYER
graph.add((pls.Player, RDFS.subClassOf, FOAF.Person))
graph.add((pls.Player, RDFS.subClassOf, schema.athlete))

#TEAM
graph.add((tms.Team, RDFS.subClassOf, schema.SportsTeam))

#PlaysIn->Team
graph.add((tms.playsIn, RDFS.domain, pls.Player))
graph.add((tms.playsIn, RDFS.range, tms.Team))
graph.add((tms.playsIn, RDF.type, RDF.Property))
graph.add((tms.playsIn, OWL.inverseOf, tms.playedBy))


#team_ranking->RANKING
graph.add((tms.team_ranking, RDFS.domain, rks.Ranking))
graph.add((tms.team_ranking, RDFS.range, tms.Team))
graph.add((tms.team_ranking, RDF.type, RDF.Property))

#home_team->Games
graph.add((tms.home_team, RDFS.domain, gms.Game))
graph.add((tms.home_team, RDFS.range, tms.Team))
graph.add((tms.home_team, RDF.type, RDF.Property))

#visitor_team->Games
graph.add((tms.visitor_team, RDFS.domain, gms.Game))
graph.add((tms.visitor_team, RDFS.range, tms.Team))
graph.add((tms.visitor_team, RDF.type, RDF.Property))

In [8]:
graph_data=graph.serialize(format='turtle')
fil=open("group07.ttl", "wb")
fil.write(graph_data)
fil.close()